In [1]:
import pandas as pd
import numpy as np
import os
import datetime
os.getcwd()

# Note email cpm updated based on the Q4 client number
# Flipp hosted not included in wide version

'/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q1'

In [2]:
TMR_up_to_2018Q4=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2018Q4/BL_MMM_long_JL_2019-04-30.csv",dtype=str)
TMR_up_to_2018Q4['impression']=TMR_up_to_2018Q4['impression'].astype(float).astype(int)
TMR_up_to_2018Q4['click']=TMR_up_to_2018Q4['click'].astype(float).astype(int)
TMR_up_to_2018Q4['cost']=TMR_up_to_2018Q4['cost'].astype(float)


In [3]:
TMR_up_to_2018Q4_no_DMA=TMR_up_to_2018Q4.groupby(["week date","media","submedia","placement"])['impression','click','cost'].sum().reset_index()
print(TMR_up_to_2018Q4_no_DMA['week date'].min())
print(TMR_up_to_2018Q4_no_DMA['week date'].max())
# TMR_up_to_2018Q4_no_DMA

2016-09-25
2019-01-27


# TV

In [4]:
TV_logs=pd.ExcelFile("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q1/input_2019Q1/Q1 2019 TMR Report 7.2_TV.xlsx")
print(TV_logs.sheet_names)
TV_logs=TV_logs.parse("Q1 2019 TMR Report 7.2",dtype=str,skiprows=1)
TV_logs.head(2)

['Q1 2019 TMR Report 7.2']


,Campaign,Status,Source,Market,Network,Media Type,Week Of,Demographic,Buy Rotation,Buy Time Period,...,Air ISCI,Cmml Title,Aired Prg Name,Est Ratings,Act Ratings,Act Impression,Rtg Source,Buy Count,Air Count,Air Detected Event
0,Q1'19,Posted,Nielsen Keeping Trac,National,BET,Cable,2019-01-28 00:00:00,W25-54,YYYYYYY,12:00AM-03:00AM,...,LGTS1031000,3FEB/BL/PRESIDENTSDAY/KASAN/$1FI,MARTIN,0.2,0.14,87454,NHI/W_0203,1,1,2140130697
1,Q1'19,Posted,Nielsen Keeping Trac,National,BET,Cable,2019-01-28 00:00:00,HH,YYYYYYY,12:00AM-03:00AM,...,LGTS1031000,3FEB/BL/PRESIDENTSDAY/KASAN/$1FI,MARTIN,0,0.2,240288,NHI/W_0203,1,1,2140130697


In [5]:
TV_logs=TV_logs[["Air Date","Vendor Net",'Act Impression','Demographic','Network']]
print(TV_logs['Demographic'].unique())
TV_logs=TV_logs[TV_logs['Demographic']=="W25-54"]
TV_logs=TV_logs[TV_logs['Air Date']!="nan"]
print(TV_logs['Demographic'].unique())

TV_logs['Vendor Net']=TV_logs['Vendor Net'].astype(float)
TV_logs['Act Impression']=TV_logs['Act Impression'].astype(float)
TV_logs['Act Impression']=TV_logs['Act Impression'].fillna(0).astype(int)
TV_logs=TV_logs.rename(columns={"Vendor Net":"cost","Act Impression":"impression"})

TV_logs['date']=TV_logs['Air Date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").date())
TV_logs=TV_logs[(TV_logs['date']<=datetime.date(2019,5,4)) & (TV_logs['date']>=datetime.date(2019,2,4))]


['W25-54' 'HH']
['W25-54']


In [6]:
TV_logs['cost'].sum()

2661712.0999999996

In [7]:
def get_week_end_date_BL(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=5-x.weekday())
    return y

In [8]:
TV_logs['week_end_dt']=TV_logs['date'].apply(lambda x: get_week_end_date_BL(x))
TV_logs['week date']=TV_logs['week_end_dt'].apply(lambda x: x-datetime.timedelta(days=6))
TV_TMR_data=TV_logs.groupby(['week date','Network'])[['cost','impression']].sum().reset_index()
TV_TMR_data=TV_TMR_data.rename(columns={"Network":"placement"})
TV_TMR_data['media']="TV"
TV_TMR_data['submedia']="National"
TV_TMR_data['click']=0
TV_TMR_data=TV_TMR_data[TMR_up_to_2018Q4_no_DMA.columns.tolist()]
TV_TMR_data['week date']=TV_TMR_data['week date'].astype(str)

In [9]:
print(TV_TMR_data['week date'].min())
print(TV_TMR_data['week date'].max())

2019-02-03
2019-04-07


In [10]:
TV_TMR_data.head(2)

,week date,media,submedia,placement,impression,click,cost
0,2019-02-03,TV,National,BET,6179321,0,52523.2
1,2019-02-03,TV,National,BRAV,481937,0,4666.5


# Others

In [11]:
Others_TMR_week=pd.read_excel("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q1/input_2019Q1/DatoramaData_2019Q1_0621.xlsx",
                              dtype=str,sheetname="BL_TMR_2019Q1")
Others_TMR_week=Others_TMR_week.rename(columns={"Week":"week date","Media":"media","Submedia":"submedia","Placement":"placement",
                                               "Impressions":"impression","Clicks":"click","Media Cost":"cost"})
Others_TMR_week['week date']=Others_TMR_week['week date'].apply(lambda x: x[:10])
Others_TMR_week['impression']=Others_TMR_week['impression'].replace("nan",0).astype(float).astype(int)
Others_TMR_week['click']=Others_TMR_week['click'].replace("nan",0).astype(float).astype(int)
Others_TMR_week['cost']=Others_TMR_week['cost'].replace("nan",0).astype(float)


### Revise Emails

In [12]:
Others_TMR_week_1=Others_TMR_week[Others_TMR_week['media']!="Email"]
Others_TMR_week_2=Others_TMR_week[Others_TMR_week['media']=="Email"]
Others_TMR_week_2['impression'].sum(),Others_TMR_week_2['cost'].sum()

(2075001231, 945753.43701974314)

In [13]:
# Based on the 2019Q1 number
# Email from Kipp, Bryan 2019 July 19, Friday 5:17 p.m.
email_cost_client=494229
email_cost_impression=2075000494
new_email_c_p_impr=email_cost_client/email_cost_impression
print(new_email_c_p_impr)
Others_TMR_week_2['cost']=Others_TMR_week_2['impression']*new_email_c_p_impr
print(new_email_c_p_impr*1000)

0.00023818259389773427
0.23818259389773427


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [14]:
print(Others_TMR_week_2['week date'].min())
print(Others_TMR_week_2['week date'].max())

Others_TMR_week=Others_TMR_week_1.append(Others_TMR_week_2)

2019-02-03
2019-04-28


### Revise Flipp

In [15]:
Others_TMR_week.head(2)

,week date,media,submedia,placement,impression,click,cost
0,2019-02-03,Digital,PLA,Google,9732670,129070,21800.330000
1,2019-02-03,Digital,SEM,Google,1791113,314094,125972.806194


In [16]:
# Remove the hosted for 2019Q1 and append the update file later
Others_TMR_week_1_nonHosted=Others_TMR_week[Others_TMR_week['placement']!="Hosted"]

Others_TMR_week_1_nonHosted['impression']=Others_TMR_week_1_nonHosted['impression'].astype(int)
Others_TMR_week_1_nonHosted['click']=Others_TMR_week_1_nonHosted['click'].astype(int)
Others_TMR_week_1_nonHosted['cost']=Others_TMR_week_1_nonHosted['cost'].astype(float)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [17]:
for col in Others_TMR_week_1_nonHosted.columns.tolist():
    print(col,Others_TMR_week_1_nonHosted[col].apply(lambda x: type(x)).unique())

week date [<class 'str'>]
media [<class 'str'>]
submedia [<class 'str'>]
placement [<class 'str'>]
impression [<class 'int'>]
click [<class 'int'>]
cost [<class 'float'>]


In [18]:
revised_Hosted_19Q1=pd.read_excel("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q1/input_2019Q1/revised_circ_2_rows_removed_in_raw.xlsx",dtype=str)
revised_Hosted_19Q1=revised_Hosted_19Q1.rename(columns={"Week":"week date","Media":"media","Submedia":"submedia","Placement":"placement",
                                               "Impressions":"impression","Clicks":"click","Media Cost":"cost"})
revised_Hosted_19Q1['impression']=revised_Hosted_19Q1['impression'].astype(int)
revised_Hosted_19Q1['click']=revised_Hosted_19Q1['click'].astype(int)
revised_Hosted_19Q1['cost']=revised_Hosted_19Q1['cost'].astype(float)
revised_Hosted_19Q1['submedia']="Flipp"
revised_Hosted_19Q1['week date']=revised_Hosted_19Q1['week date'].apply(lambda x: x.split("-")[0][:-1])
revised_Hosted_19Q1['week date']=revised_Hosted_19Q1['week date'].apply(lambda x: datetime.datetime.strptime(x,"%d %b %Y"))
revised_Hosted_19Q1['week date']=revised_Hosted_19Q1['week date'].astype(str)

revised_Hosted_19Q1=revised_Hosted_19Q1[Others_TMR_week_1_nonHosted.columns.tolist()]

In [19]:
Others_TMR_week=Others_TMR_week_1_nonHosted.append(revised_Hosted_19Q1)

### Revise Circ

In [20]:
# Not finished yet
updated_circ_2019Q1_Kayla=pd.read_excel("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q1/input_2019Q1/BL_circ_variance_JL_20190910.xlsx",
                                        dtype=str,skiprows=1)
updated_circ_2019Q1_Kayla['AD Date']=updated_circ_2019Q1_Kayla['AD Date'].apply(lambda x: x.split(" ")[0])
updated_circ_2019Q1_Kayla=updated_circ_2019Q1_Kayla.iloc[:,0:3]

print(pd.to_datetime(updated_circ_2019Q1_Kayla['AD Date'],format="%Y-%m-%d").dt.weekday_name.unique())
updated_circ_2019Q1_Kayla=updated_circ_2019Q1_Kayla.rename(columns={"AD Date":"week date","Quantity":"impression","Cost":"cost"})
updated_circ_2019Q1_Kayla['click']=0
updated_circ_2019Q1_Kayla['media']="Circulation"
updated_circ_2019Q1_Kayla['submedia']="xx"
updated_circ_2019Q1_Kayla['placement']="xx"
updated_circ_2019Q1_Kayla['impression']=updated_circ_2019Q1_Kayla['impression'].astype(int)
updated_circ_2019Q1_Kayla['cost']=updated_circ_2019Q1_Kayla['cost'].astype(int)

updated_circ_2019Q1_Kayla=updated_circ_2019Q1_Kayla[Others_TMR_week.columns.tolist()]


['Sunday']


In [21]:
Others_TMR_week=Others_TMR_week[Others_TMR_week['media']!="Circulation"]
Others_TMR_week=Others_TMR_week.append(updated_circ_2019Q1_Kayla)

In [22]:
for col in Others_TMR_week.columns.tolist():
    print(col,Others_TMR_week[col].apply(type).unique())

week date [<class 'str'>]
media [<class 'str'>]
submedia [<class 'str'>]
placement [<class 'str'>]
impression [<class 'int'>]
click [<class 'int'>]
cost [<class 'float'>]


# Cumulative Q1 TMR

In [23]:
Q1_2019_only_TMR=Others_TMR_week.append(TV_TMR_data)

Cum_Q1_2019_TMR=TMR_up_to_2018Q4.append(Q1_2019_only_TMR)

In [24]:
Q1_2019_only_TMR.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q1/output/BL_MMM_2019Q1_Only_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
Cum_Q1_2019_TMR.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q1/output/BL_MMM_long_cumu_up_to_2019Q1_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [25]:
Cum_Q1_2019_TMR[Cum_Q1_2019_TMR['media']=="Circulation"]['submedia'].unique()

array(['FSI+Instore', 'Valassis', 'xx'], dtype=object)

In [26]:
Cum_Q1_2019_TMR.head(2)

,cleaned dma,click,cost,impression,media,placement,submedia,week date
0,"Los Angeles, CA",6687,309.818159,679747,Email,xx,xx,2016-09-25
1,"Dallas-Ft. Worth, TX",3185,147.561605,323753,Email,xx,xx,2016-09-25


# Wide

In [27]:
# seperate the hosted flipp
Cum_Q1_2019_TMR_FlippHost=Cum_Q1_2019_TMR[(Cum_Q1_2019_TMR['submedia']=="Flipp") & (Cum_Q1_2019_TMR['placement']=="Hosted")]
Cum_Q1_2019_TMR_FlippHost['submedia']="Flipp(hosted)"

Cum_Q1_2019_TMR_FlippNonHost=Cum_Q1_2019_TMR[(Cum_Q1_2019_TMR['submedia']=="Flipp") & (Cum_Q1_2019_TMR['placement']!="Hosted")]
Cum_Q1_2019_TMR_FlippNonHost['submedia']="Flipp(nonhosted)"


Cum_Q1_2019_TMR=Cum_Q1_2019_TMR[(Cum_Q1_2019_TMR['submedia']!="Flipp")]
Cum_Q1_2019_TMR['submedia']=np.where(Cum_Q1_2019_TMR['media']=="Circulation","xx",Cum_Q1_2019_TMR['submedia'])


Cum_Q1_2019_TMR=Cum_Q1_2019_TMR.append(Cum_Q1_2019_TMR_FlippHost).append(Cum_Q1_2019_TMR_FlippNonHost)


########################

data_media_national=Cum_Q1_2019_TMR.groupby(['week date','media'])['impression','click','cost'].sum().reset_index()
data_submedia_national=Cum_Q1_2019_TMR.groupby(['week date','media','submedia'])['impression','click','cost'].sum().reset_index()
data_submedia_national['media_submedia']=data_submedia_national['media']+"-"+data_submedia_national['submedia']
data_submedia_national=data_submedia_national[['week date','media_submedia','impression','click','cost']]


data_media_national_wide=data_media_national[['week date']].drop_duplicates()
for col in data_media_national.columns.tolist()[-3:]:
    df=data_media_national[['week date','media']+[col]].pivot(index='week date',columns='media',values=col).reset_index()
    df_new_col_list=df.columns.tolist()
    df_new_col_list.remove('week date')
    df_new_col_dict={}
    for new_col in df_new_col_list:
        df_new_col_dict.update({new_col:new_col+"_"+col})
    df=df.rename(columns=df_new_col_dict)
    data_media_national_wide=pd.merge(data_media_national_wide,df,on='week date',how="outer")
data_media_national_wide=data_media_national_wide.fillna(0)
data_media_national_wide=data_media_national_wide.drop_duplicates()
#############

data_submedia_national_wide=data_submedia_national[['week date']].drop_duplicates()
for col in data_submedia_national.columns.tolist()[-3:]:
    df=data_submedia_national[['week date','media_submedia']+[col]].pivot(index='week date',columns='media_submedia',values=col).reset_index()
    df_new_col_list=df.columns.tolist()
    df_new_col_list.remove('week date')
    df_new_col_dict={}
    for new_col in df_new_col_list:
        df_new_col_dict.update({new_col:new_col+"_"+col})
    df=df.rename(columns=df_new_col_dict)
    data_submedia_national_wide=pd.merge(data_submedia_national_wide,df,on='week date',how="outer")
data_submedia_national_wide=data_submedia_national_wide.fillna(0)
data_submedia_national_wide=data_submedia_national_wide.drop_duplicates()


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [28]:
Cum_Q1_2019_TMR['click'].apply(lambda x: type(x)).unique()

array([<class 'int'>], dtype=object)

In [29]:
including_weeks=[str(datetime.date(2019,5,4)-datetime.timedelta(days=7*x+6)) for x in range(104)]


data_media_national_wide=data_media_national_wide.sort_values("week date")
data_media_national_wide=data_media_national_wide[data_media_national_wide['week date'].isin(including_weeks)]

data_submedia_national_wide=data_submedia_national_wide.sort_values("week date")
data_submedia_national_wide=data_submedia_national_wide[data_submedia_national_wide['week date'].isin(including_weeks)]



## Sales

In [36]:
sales_data=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-07-13.csv",dtype=str)
sales_data=sales_data[sales_data['week_end_date']<="2019-05-04"]
sales_data['week_end_date']=sales_data['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
sales_data['sales']=sales_data['sales'].astype(float)
sales_data['transactions']=sales_data['transactions'].astype(float)
sales_data['week date']=sales_data['week_end_date'].apply(lambda x: x-datetime.timedelta(days=6))
sales_data_naitonal=sales_data.groupby(['week date'])['sales','transactions'].sum().reset_index()
sales_data_naitonal=sales_data_naitonal.set_index(['week date'])
sales_data_naitonal.at[datetime.date(2017,4,30),'transactions']=(sales_data_naitonal.at[datetime.date(2017,4,23),'transactions']+sales_data_naitonal.at[datetime.date(2017,5,7),'transactions'])/2
sales_data_naitonal=sales_data_naitonal.reset_index()
sales_data_store_counts=sales_data[sales_data['sales']>0]
sales_data_store_counts=sales_data_store_counts.groupby(['week date'])['location_id'].count().to_frame().reset_index().rename(columns={"location_id":"store_counts"})

sales_data_naitonal=pd.merge(sales_data_naitonal,sales_data_store_counts,on="week date",how="outer")
sales_data_naitonal['week date']=sales_data_naitonal['week date'].astype(str)

In [37]:
data_media_national_wide=pd.merge(data_media_national_wide,sales_data_naitonal,on="week date",how="left")
data_submedia_national_wide=pd.merge(data_submedia_national_wide,sales_data_naitonal,on="week date",how="left")

## Promotion

In [38]:
Rewards_Promotion_list=[datetime.date(2016,10,1),datetime.date(2016,10,2),
                        datetime.date(2017,1,21),datetime.date(2017,1,22),
                        datetime.date(2017,4,1),datetime.date(2017,4,2),
                        datetime.date(2017,7,8),datetime.date(2017,7,9),
                        datetime.date(2017,9,30),datetime.date(2017,10,1),
                        datetime.date(2018,1,20),datetime.date(2018,1,21),
                        datetime.date(2018,4,7),datetime.date(2018,4,8),
                        datetime.date(2018,7,7),datetime.date(2018,7,8),
                        datetime.date(2018,9,29),datetime.date(2018,9,30),
                        datetime.date(2019,1,26),datetime.date(2019,1,27),  # Changed here
                        datetime.date(2019,4,6),datetime.date(2019,4,7)]

df_Rewards_Promotion=pd.DataFrame({"Date":Rewards_Promotion_list},index=range(len(Rewards_Promotion_list)))
df_Rewards_Promotion['weekday']=df_Rewards_Promotion['Date'].apply(lambda x: x.weekday())
df_Rewards_Promotion['week date']=np.where(df_Rewards_Promotion['weekday']==6,df_Rewards_Promotion['Date'],df_Rewards_Promotion['Date']-datetime.timedelta(days=6))

del df_Rewards_Promotion['Date']

df_Rewards_Promotion_Sunday=df_Rewards_Promotion[df_Rewards_Promotion['weekday']==6]
df_Rewards_Promotion_Sunday['Sunday_rewards_ind']=1
del df_Rewards_Promotion_Sunday['weekday']
df_Rewards_Promotion_Sunday['week date']=df_Rewards_Promotion_Sunday['week date'].astype(str)


df_Rewards_Promotion_Saturday=df_Rewards_Promotion[df_Rewards_Promotion['weekday']==5]
df_Rewards_Promotion_Saturday['Saturday_rewards_ind']=1
del df_Rewards_Promotion_Saturday['weekday']
df_Rewards_Promotion_Saturday['week date']=df_Rewards_Promotion_Saturday['week date'].astype(str)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [39]:
holiday_1_week_df=pd.DataFrame({"week date":[datetime.date(2016,12,18),datetime.date(2017,12,17),datetime.date(2018,12,23)],"Holiday_1_week_only_Ind":[1]*3},index=[0,1,2])
holiday_5_weeks_df=pd.DataFrame({"week date":[datetime.date(2016,12,18)-datetime.timedelta(days=x*7) for x in range(5)] +[datetime.date(2017,12,17)-datetime.timedelta(days=x*7) for x in range(5)]+[datetime.date(2018,12,23)-datetime.timedelta(days=x*7) for x in range(5)],
                                 "Holiday_5_weeks_only_Ind":[1]*15},index=[x for x in range(15)])

holiday_1_week_df['week date']=holiday_1_week_df['week date'].astype(str)
holiday_5_weeks_df['week date']=holiday_5_weeks_df['week date'].astype(str)


df_special_weeks_ind=pd.merge(df_Rewards_Promotion_Saturday,df_Rewards_Promotion_Sunday,on="week date",how="outer")
df_special_weeks_ind=pd.merge(df_special_weeks_ind,holiday_1_week_df,on="week date",how="outer")
df_special_weeks_ind=pd.merge(df_special_weeks_ind,holiday_5_weeks_df,on="week date",how="outer")


In [40]:
data_media_national_wide=pd.merge(data_media_national_wide,df_special_weeks_ind,on="week date",how="left")

data_submedia_national_wide=pd.merge(data_submedia_national_wide,df_special_weeks_ind,on="week date",how="left")

data_media_national_wide=data_media_national_wide.fillna(0)
data_submedia_national_wide=data_submedia_national_wide.fillna(0)

In [41]:
data_media_national_wide.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q1/output/BL_MMM_data_media_national_wide_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
data_submedia_national_wide.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q1/output/BL_MMM_data_submedia_national_wide_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

# Note email cpm updated based on the Q1 client number
# Flipp hosted seperated in wide version
# Circ # is based on internal and different from Dom's 
